In [1]:
glider_names = [f"ifm{n:02d}" for n in range(1, 16)]
max_depl_num = 99

In [2]:
print(glider_names)

['ifm01', 'ifm02', 'ifm03', 'ifm04', 'ifm05', 'ifm06', 'ifm07', 'ifm08', 'ifm09', 'ifm10', 'ifm11', 'ifm12', 'ifm13', 'ifm14', 'ifm15']


In [3]:
import requests
import re
import pandas as pd
from time import sleep

In [4]:
t, lat, lon = requests.get("https://data.geomar.de/realtime/data/project/ifm14/ifm14_pos.json").json()

In [5]:
def read_glider_pos_json(glider_name):
    url = f"https://data.geomar.de/realtime/data/project/{glider_name}/{glider_name}_pos.json"
    try:
        sleep(0.5)
        glider_pos_data = requests.get(url, timeout=1).json()
    except:
        print(f"failed for {glider_name}")
        sleep(2.0)
        try:
            
            glider_pos_data = requests.get(url, timeout=5).json()
        except:
            print(f"failed for {glider_name}")
            glider_pos_data = [None, None, None]
    return {
        "time": glider_pos_data[0],
        "latitude": glider_pos_data[1],
        "longitude": glider_pos_data[2],
    }

In [6]:
%%time

positions = {
    glider_name: read_glider_pos_json(glider_name)
    for glider_name in glider_names
}

failed for ifm01
failed for ifm01
failed for ifm02
failed for ifm04
failed for ifm04
failed for ifm05
failed for ifm05
failed for ifm06
failed for ifm06
failed for ifm10
failed for ifm10
CPU times: user 585 ms, sys: 95.9 ms, total: 680 ms
Wall time: 32.5 s


In [7]:
positions = pd.DataFrame.from_dict(positions, orient="index")

In [8]:
positions["platform"] = positions.index

In [9]:
positions["time"] = positions["time"].apply(pd.Timestamp)

In [10]:
positions

,time,latitude,longitude,platform
ifm01,NaT,None,None,ifm01
ifm02,2018-08-26 21:46:07,27.7938,-90.0739,ifm02
ifm03,2020-02-03 16:41:59,12.0177,-59.2430,ifm03
ifm04,NaT,None,None,ifm04
ifm05,NaT,None,None,ifm05
ifm06,NaT,None,None,ifm06
ifm07,2017-06-10 16:15:52,-12.5398,-77.9009,ifm07
ifm08,2019-12-10 08:13:33,17.8038,-22.4636,ifm08
ifm09,2020-02-09 14:51:30,13.8331,-57.3889,ifm09
ifm10,NaT,None,None,ifm10


In [11]:
positions["time"].apply(pd.Timestamp)

ifm01                   NaT
ifm02   2018-08-26 21:46:07
ifm03   2020-02-03 16:41:59
ifm04                   NaT
ifm05                   NaT
ifm06                   NaT
ifm07   2017-06-10 16:15:52
ifm08   2019-12-10 08:13:33
ifm09   2020-02-09 14:51:30
ifm10                   NaT
ifm11   2019-11-22 11:24:27
ifm12   2020-02-17 11:56:12
ifm13   2020-01-24 18:26:49
ifm14   2019-12-10 16:02:50
ifm15   2019-12-18 16:20:26
Name: time, dtype: datetime64[ns]

In [12]:
!mkdir -p data/
positions.dropna().to_csv("data/gliders.csv")

In [13]:
!cat data/gliders.csv

,time,latitude,longitude,platform
ifm02,2018-08-26 21:46:07,27.7938,-90.0739,ifm02
ifm03,2020-02-03 16:41:59,12.0177,-59.2430,ifm03
ifm07,2017-06-10 16:15:52,-12.5398,-77.9009,ifm07
ifm08,2019-12-10 08:13:33,17.8038,-22.4636,ifm08
ifm09,2020-02-09 14:51:30,13.8331,-57.3889,ifm09
ifm11,2019-11-22 11:24:27,16.8883,-24.9984,ifm11
ifm12,2020-02-17 11:56:12,12.5908,-57.3730,ifm12
ifm13,2020-01-24 18:26:49,11.5479,-57.9964,ifm13
ifm14,2019-12-10 16:02:50,17.5700,-21.1419,ifm14
ifm15,2019-12-18 16:20:26,14.6628,-25.5229,ifm15
